# Instalação e importação das bibliotecas

In [1]:
!pip install opencv-python-headless numpy tensorflow matplotlib

In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import random
from tensorflow.keras.applications import ResNet50

2024-10-09 02:27:13.784336: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-09 02:27:13.799735: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-09 02:27:13.950737: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-09 02:27:14.081814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 02:27:14.198561: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

**Atenção:** O aviso acima apenas informa que não foram encontrados drivers cuda e por conta disso será usada a CPU da máquina ao invés da GPU

# Coleta e processamento de dados

In [3]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0
    return img_normalized


# Balanceamento do DataSet
Para evitar maior peso em um determinado tipo de dado, foi feito um balanceamento do dataset.

Para isso, foi utilizado o método de reamostragem chamado de undersampling, no qual foi selecionado aleatoriamente os dados de uma classe e duplicado até que a quantidade de dados de cada classe se iguale.

In [4]:
dataset_dir = './dataset'
balanced_dataset_dir = './balanced-dataset'

classes = ['healthy', 'kyphosis', 'lordosis']

os.makedirs(balanced_dataset_dir, exist_ok=True)

image_counts = {}
for cls in classes:
    cls_path = os.path.join(dataset_dir, cls)
    image_counts[cls] = len(os.listdir(cls_path))

max_images = max(image_counts.values())
new_image_counts = {}

for cls in classes:
    os.makedirs(os.path.join(balanced_dataset_dir, cls), exist_ok=True)
    
    images = os.listdir(os.path.join(dataset_dir, cls))
    
    for index, image in enumerate(images):
        new_name = f"{cls}_{index + 1:03d}.jpg"
        shutil.copy(os.path.join(dataset_dir, cls, image), 
                    os.path.join(balanced_dataset_dir, cls, new_name))
    
    new_image_counts[cls] = len(images)

    current_count = new_image_counts[cls]
    while current_count < max_images:
        image_to_copy = random.choice(images)
        
        new_name = f"{cls}_{current_count + 1:03d}.jpg"
        shutil.copy(os.path.join(dataset_dir, cls, image_to_copy),
                    os.path.join(balanced_dataset_dir, cls, new_name))
        
        current_count += 1

print("Dataset balanceado criado em:", balanced_dataset_dir)


Dataset balanceado criado em: ./balanced-dataset


### Criando CNN (rede neural convolucional)
Essa função cria um modelo CNN que passa por algumas camadas convolucionais para extrair características das imagens, nas quais foram utilizados as funções:
- ReLU 
	- Função de ativação que permite que a rede neural aprenda padrões complexos
- Pooling
	- Operação de amostragem usada em redes convolucionais para reduzir as dimensões e/ou tamanho da imagem ou das saídas das camadas convolucionais, porém, preservando suas características mais importantes
- Normalização de Batch
	- Normalização de dados de entrada para que a rede neural possa aprender mais rapidamente
- Softmax
	- Função de ativação que converte um vetor de valores reais em uma distribuição de probabilidade

In [19]:
def create_3_class_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    # Converte a imagem grayscale para 3 canais
    x = layers.Conv2D(3, (1, 1), padding="same")(inputs)
    
    # Base do modelo ResNet50 com pesos da ImageNet
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    # Passa pela base do modelo
    x = base_model(x)
    
    # Adicionando mais convoluções e pooling
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)

    # Verifique o tamanho da saída após as convoluções e pooling
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(3, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)  # Criar o modelo corretamente
    
    return model

def create_2_class_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    # Converte a imagem grayscale para 3 canais
    x = layers.Conv2D(3, (1, 1), padding="same")(inputs)
    
    # Base do modelo ResNet50 com pesos da ImageNet
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    # Passa pela base do modelo
    x = base_model(x)
    
    # Adicionando mais convoluções e pooling
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)

    # Verifique o tamanho da saída após as convoluções e pooling
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)  # Criar o modelo corretamente
    
    return model

# Define o shape de entrada com 1 canal (grayscale)
input_shape = (224, 224, 1)

# Criação dos modelos
generalist_cnn_model = create_3_class_cnn(input_shape)
lordosis_cnn_model = create_2_class_cnn(input_shape)
kiphosis_cnn_model = create_2_class_cnn(input_shape)

lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)


# Pré-processamento dos modelos

In [20]:
train_datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    brightness_range = [0.8, 1.2],
    validation_split = 0.3
)

In [21]:
generalist_classes = ['healthy', 'kyphosis', 'lordosis']

generalist_train_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'training'
)

generalist_validation_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'validation'
)

generalist_cnn_model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

Found 36 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [22]:
lordosis_classes = ['healthy', 'lordosis']

lordosis_train_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'training',
    classes = lordosis_classes
)

lordosis_validation_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'validation',
    classes = lordosis_classes
)

lordosis_cnn_model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

Found 24 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [23]:
kiphosis_classes = ['healthy', 'kyphosis']

kiphosis_train_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'training',
    classes = kiphosis_classes
)

kiphosis_validation_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'validation',
    classes = kiphosis_classes
)

kiphosis_cnn_model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

Found 24 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


# Treinamento do modelo

In [24]:
generalist_history = generalist_cnn_model.fit(
    generalist_train_generator,
    validation_data = generalist_validation_generator,
    epochs = 30,
    callbacks = [lr_scheduler]
)

lordosis_history = lordosis_cnn_model.fit(
    lordosis_train_generator,
    validation_data = lordosis_validation_generator,
    epochs = 30,
    callbacks = [lr_scheduler]
)

kiphosis_history = kiphosis_cnn_model.fit(
    kiphosis_train_generator,
    validation_data = kiphosis_validation_generator,
    epochs = 30,
    callbacks = [lr_scheduler]
)

/opt/conda/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.3043 - loss: 4.7112 - val_accuracy: 0.2667 - val_loss: 4.1938 - learning_rate: 0.0010
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.3930 - loss: 4.1567 - val_accuracy: 0.4667 - val_loss: 3.8413 - learning_rate: 0.0010
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.4428 - loss: 4.1630 - val_accuracy: 0.4667 - val_loss: 3.6349 - learning_rate: 0.0010
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.5900 - loss: 3.6679 - val_accuracy: 0.2000 - val_loss: 3.8179 - learning_rate: 0.0010
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.5023 - loss: 3.9634 - val_accuracy: 0.4667 - val_loss: 3.6675 - learning_rate: 0.0010
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.4279 - loss: 3.9559 - val_accuracy: 0.3333 - val_loss: 3.5652 - learning_rate: 0.0010
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.7073 - loss: 3.2250 - val_accuracy: 0.6000 - val_l

## Porque resultados sempre são diferentes:
- Ao inicializar o modelo os pesos da rede neural são inicializados de forma aleatória, e como o processo de otimização do modelo começa a partir de diferentes pontos cada execução pode levar a resultados diferentes, por isso, cada vez que o código for executado os resultados não serão exatamente os mesmos, mas aproximados.
- Image Augmentation: técnicas de rotação, deslocamento, zoom e etc. aplicadas na imagem para criar novas variações para realizar o treinamento do modelo, sendo também um processo aleatório.
- Shuffling: Durante o treinamento, os dados de treino são embaralhados a cada época. Isso garante que o modelo não aprenda de forma dependente da ordem dos exemplos, mas também pode fazer com que os resultados variem.

# Validação dos modelos

In [25]:
generalist_val_loss, generalist_val_acc = generalist_cnn_model.evaluate(generalist_validation_generator)
lordosis_val_loss, lordosis_val_acc = lordosis_cnn_model.evaluate(lordosis_validation_generator)
kiphosis_val_loss, kiphosis_val_acc = kiphosis_cnn_model.evaluate(kiphosis_validation_generator)

print(f"Validação modelo generalista - Loss: {generalist_val_loss}, Acurácia: {generalist_val_acc}")
print(f"Validação modelo de lordose - Loss: {lordosis_val_loss}, Acurácia: {lordosis_val_acc}")
print(f"Validação modelo de cifose - Loss: {kiphosis_val_loss}, Acurácia: {kiphosis_val_acc}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 926ms/step - accuracy: 0.5567 - loss: 2.9815
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 958ms/step - accuracy: 0.8188 - loss: 2.6659
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 697ms/step - accuracy: 0.6313 - loss: 2.4899
Validação modelo generalista - Loss: 3.0154881477355957, Acurácia: 0.6000000238418579
Validação modelo de lordose - Loss: 2.738570213317871, Acurácia: 0.699999988079071
Validação modelo de cifose - Loss: 2.4555375576019287, Acurácia: 0.699999988079071


# Função de predição

In [26]:
def predict_image(image_path, model):
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    return predicted_class, prediction

generalist_predicted_class, generalist_result = predict_image('./test-data/001.jpg', generalist_cnn_model)
lordosis_predicted_class, lordosis_result = predict_image('./test-data/001.jpg', lordosis_cnn_model)
kiphosis_predicted_class, kiphosis_result = predict_image('./test-data/001.jpg', kiphosis_cnn_model)

print(f'Predição (modelo generalista): {generalist_result}')
print(f'Classe prevista (modelo generalista): {generalist_classes[generalist_predicted_class]}')

print(f'Predição (modelo de lordose): {lordosis_result}')
print(f'Classe prevista (modelo de lordose): {lordosis_classes[lordosis_predicted_class]}')

print(f'Predição (modelo de cifose): {kiphosis_result}')
print(f'Classe prevista (modelo de cifose): {kiphosis_classes[kiphosis_predicted_class]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predição (modelo generalista): [[0.03289848 0.08524974 0.8818518 ]]
Classe prevista (modelo generalista): lordosis
Predição (modelo de lordose): [[0.41304484 0.58695513]]
Classe prevista (modelo de lordose): lordosis
Predição (modelo de cifose): [[0.6733316  0.32666835]]
Classe prevista (modelo de cifose): healthy
